In [22]:
""" import """
import os
from dotenv import load_dotenv

import openai

from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.chat_models import ChatOpenAI

from langchain.chains.router import MultiPromptChain
from langchain.chains import ConversationChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.router.embedding_router import EmbeddingRouterChain
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import Chroma

In [27]:
""" set OpenAI API key """
load_dotenv(dotenv_path="../../../.env")
openai.api_key = os.getenv("OPENAI_API_KEY")
cohere_api_key = os.getenv("COHERE_API_KEY")

llm = OpenAI(openai_api_key=openai.api_key, temperature=0.9)
chat = ChatOpenAI(openai_api_key=openai.api_key, temperature=0.9)

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
]

In [ ]:
destination_chains = {}

for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain


default_chain = ConversationChain(llm=llm, output_key="text")

In [ ]:
""" Uses an LLM to determine how to route things. """

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

print(destinations_str) # print the destinations

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
 
print(router_template) # print the router template

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
"""
 MultiPromptChain to create a question-answering chain.
 It selects the prompt which is most relevant for a given question, and then answers the question using that prompt.
"""

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [ ]:
""" Chat with the chain """
print(chain.run("What is black body radiation?")) # physics
print(chain.run("What is the first prime number greater than 40 such that one plus the prime number is divisible by 3")) # math
print(chain.run("What is the name of the type of cloud that rins")) # none

In [24]:
"""
EmbeddingRouterChain uses embeddings and similarity to route between destination chains.
"""

names_and_descriptions = [
    ("physics", ["for questions about physics"]),
    ("math", ["for questions about math"]),
]


router_chain = EmbeddingRouterChain.from_names_and_descriptions(
    names_and_descriptions, Chroma, CohereEmbeddings(model="embed-english-light-v2.0", cohere_api_key=cohere_api_key), routing_keys=["input"]
)

In [25]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [26]:
""" Chat with the chain """
print(chain.run("What is black body radiation?"))
print(chain.run("What is the first prime number greater than 40 such that one plus the prime number is divisible by 3"))



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


Black body radiation is the thermal electromagnetic radiation that is emitted from an object that is at a temperature above absolute zero. The radiation is made up of a broad range of wavelengths as the object's temperature increases. Black body radiation can be described by Planck's law which states that the power emitted by black body radiation is proportional to the fourth power of its absolute temperature.


> Entering new MultiPromptChain chain...
math: {'input': 'What is the first prime number greater than 40 such that one plus the prime number is divisible by 3'}
> Finished chain.
?

The first prime number greater than 40 such that one plus the prime number is divisible by 3 is 43. To prove this, we need to look at the prime numbers from 41 to 43 to see which one satisfies the condition. 41 is not prime, 42 is not prime, and 43 is prime, so 43 is the answer to the q